In [2]:
import numpy
import urllib
import scipy.optimize
import random
from sklearn import svm

In [3]:
def parseData(fname):
  for l in urllib.urlopen(fname):
    yield eval(l)

In [4]:
    print "Reading data..."
    data = list(parseData("beer_50000.json"))
    print "done"
    print 'Question 1:'
    X = [d['review/taste'] for d in data]
    sum1 = 0
    max1 = 0    
    for x in X:
        sum1 += x
        if max1 < x:
            max1 = x
    mean = sum1/len(X)
    print 'max1', max1
    print 'mean: ', mean
    sum2  = float(0)
    for x in X:
        sum2 += (x-mean)**2
    var = sum2/len(X)
    print 'var: ',var

Reading data...
done
Question 1:
max1 5.0
mean:  3.92225
var:  0.5135699375


In [6]:
    print 'Question 2:'
    bs_count = {}
    beer_styles = [(d['beer/style'],d['review/taste']) for d in data]
    for bs in beer_styles:
        if bs[0] in bs_count.keys():
            bs_count[bs[0]].append(bs[1])
        else:
            bs_count[bs[0]] = list([bs[1]])
    for style in bs_count.keys():
        print style,":", len(bs_count[style])
        sum1 = 0
        for rate in bs_count[style]:
            sum1 += rate
        print "avg review for ",style,":", float(sum1)/len(bs_count[style])

Question 2:
Tripel : 257
avg review for  Tripel : 3.78404669261
American Black Ale : 138
avg review for  American Black Ale : 3.8731884058
Fruit / Vegetable Beer : 1355
avg review for  Fruit / Vegetable Beer : 3.60774907749
English Brown Ale : 495
avg review for  English Brown Ale : 3.72828282828
American Pale Ale (APA) : 2288
avg review for  American Pale Ale (APA) : 3.64969405594
American Amber / Red Lager : 42
avg review for  American Amber / Red Lager : 3.69047619048
Chile Beer : 11
avg review for  Chile Beer : 3.95454545455
Cream Ale : 69
avg review for  Cream Ale : 3.02898550725
Dubbel : 165
avg review for  Dubbel : 3.73636363636
English Dark Mild Ale : 21
avg review for  English Dark Mild Ale : 3.78571428571
Belgian Strong Pale Ale : 632
avg review for  Belgian Strong Pale Ale : 4.05617088608
Weizenbock : 13
avg review for  Weizenbock : 3.38461538462
Milk / Sweet Stout : 69
avg review for  Milk / Sweet Stout : 3.78260869565
Dunkelweizen : 61
avg review for  Dunkelweizen : 3.4918

In [6]:
    def feature1(datum):
      feat = [1]
      if datum['beer/style'] == "American IPA":
        feat.append(1)
      else:
        feat.append(0)
      return feat

    print 'Question 3:'
    X = [feature1(d) for d in data]
    y = [d['review/taste'] for d in data]
    theta,residuals,rank,s = numpy.linalg.lstsq(X, y) 
    print 'theta: ', theta

Question 3:
theta:  [ 3.91520474  0.08564622]


In [7]:
    print 'Question 4:'
    # train
    X = list()
    Y = list()
    Xt = list()
    Yt = list()
    count = 0
    for d in data:
        if count < 25000:
            Xt.append(feature1(d))
            Yt.append(d['review/taste'])
        else:
            X.append(feature1(d))
            Y.append(d['review/taste'])
        count+=1
    theta,res,rank,s = numpy.linalg.lstsq(Xt, Yt)
    ytest = numpy.matmul(X,theta)
    diff = [(ytest[i] - Y[i])**2 for i in range(25000)]
    print 'MSE(train): ', res/len(Yt)
    print 'MSE(test): ', sum(diff)/len(ytest)
    

Question 4:
MSE(train):  [ 0.55810729]
MSE(test):  0.468410050967


In [8]:
    print 'Question 5:' 
    beers = {}
    Y = list()
    # prepare Y
    for d in data:
        Y.append(d['review/taste'])
        if d['beer/style'] not in beers.keys():
            beers[d['beer/style']] = list()
    print 'Y is ready'
    # prepare X
    for d in data:
        beers[d['beer/style']].append([1,1])
        for b in beers.keys():
            if b != d['beer/style']:
                beers[b].append([1,0])
    print 'X is ready'    
    for b in beers.keys():
        xtrain = beers[b][:25000]
        xtest = beers[b][25000:]
        theta,res,rank,s = numpy.linalg.lstsq(xtrain, Y[:25000])
        Y_test = numpy.matmul(xtest,theta)
        Y_real = Y[25000:]
        diff = [(Y_test[i] - Y_real[i])**2 for i in range(25000)]
        print b,'\'s thetas:', theta
        print b,'\'s MSE(train): ', res/25000
        print b,'\'s MSE(test): ', sum(diff)/len(Y_real)
        print '-------------------------------------'

Question 5:
Y is ready
X is ready
Tripel 's thetas: [ 3.91103454 -0.24525914]
Tripel 's MSE(train):  [ 0.55790879]
Tripel 's MSE(test):  0.469460683188
-------------------------------------
American Black Ale 's thetas: [ 3.90948103 -0.10645073]
American Black Ale 's MSE(train):  [ 0.55832552]
American Black Ale 's MSE(test):  0.469160344682
-------------------------------------
Fruit / Vegetable Beer 's thetas: [ 3.91633805 -0.19631605]
Fruit / Vegetable Beer 's MSE(train):  [ 0.557005]
Fruit / Vegetable Beer 's MSE(test):  0.465726670135
-------------------------------------
English Brown Ale 's thetas: [ 3.91088392 -0.26985827]
English Brown Ale 's MSE(train):  [ 0.55790378]
English Brown Ale 's MSE(test):  0.468985032804
-------------------------------------
American Pale Ale (APA) 's thetas: [ 3.92184427 -0.28763135]
American Pale Ale (APA) 's MSE(train):  [ 0.55487835]
American Pale Ale (APA) 's MSE(test):  0.465505293826
-------------------------------------
American Amber / Red

Belgian Dark Ale 's thetas: [ 3.91321598 -0.57700909]
Belgian Dark Ale 's MSE(train):  [ 0.55605423]
Belgian Dark Ale 's MSE(test):  0.468948937389
-------------------------------------
Rauchbier 's thetas: [ 3.90891784  0.14108216]
Rauchbier 's MSE(train):  [ 0.55831563]
Rauchbier 's MSE(test):  0.46724624971
-------------------------------------
Hefeweizen 's thetas: [ 3.91296866 -0.31197528]
Hefeweizen 's MSE(train):  [ 0.55719383]
Hefeweizen 's MSE(test):  0.46823863264
-------------------------------------
Euro Strong Lager 's thetas: [ 3.91872378 -1.14468532]
Euro Strong Lager 's MSE(train):  [ 0.54754433]
Euro Strong Lager 's MSE(test):  0.464605861842
-------------------------------------
Baltic Porter 's thetas: [ 3.90890348  0.35300128]
Baltic Porter 's MSE(train):  [ 0.55825078]
Baltic Porter 's MSE(test):  0.467392743466
-------------------------------------
Pumpkin Ale 's thetas: [ 3.90969377 -0.03578073]
Pumpkin Ale 's MSE(train):  [ 0.55833794]
Pumpkin Ale 's MSE(test): 

English Strong Ale 's thetas: [ 3.9103111  -0.18155293]
English Strong Ale 's MSE(train):  [ 0.55815487]
English Strong Ale 's MSE(test):  0.469118973612
-------------------------------------
American Blonde Ale 's thetas: [ 3.91539299 -0.76268363]
American Blonde Ale 's MSE(train):  [ 0.55367042]
American Blonde Ale 's MSE(test):  0.467482995489
-------------------------------------
Witbier 's thetas: [ 3.91155286 -0.39477433]
Witbier 's MSE(train):  [ 0.55743205]
Witbier 's MSE(test):  0.468983071787
-------------------------------------
American Stout 's thetas: [ 3.91127945 -0.10133793]
American Stout 's MSE(train):  [ 0.55814896]
American Stout 's MSE(test):  0.469036076076
-------------------------------------
Irish Red Ale 's thetas: [ 3.9122191 -0.9434691]
Irish Red Ale 's MSE(train):  [ 0.55551605]
Irish Red Ale 's MSE(test):  0.468952375366
-------------------------------------
Eisbock 's thetas: [ 3.90925096 -0.15925096]
Eisbock 's MSE(train):  [ 0.55834725]
Eisbock 's MSE(t

In [5]:
    print 'Question 6:'
    ran = 25000
    c = 1000
    X = list()
    Y = list()
    for d in data:
        X.append([d['beer/ABV'],d['review/taste']])
        if d['beer/style'] == 'American IPA':
            Y.append(1)
        else:
            Y.append(0)
    print 'XY ready'
    X_train = X[:ran]
    y_train = Y[:ran]
    X_test = X[ran:2*ran]
    y_test = Y[ran:2*ran]
    # Create a support vector classifier object, with regularization parameter C = 1000
    clf = svm.SVC(C=c)
    print 'c = ',c
    clf.fit(X_train, y_train) 
    train_predictions = clf.predict(X_train)
    correct = 0 
    for i in range(0,ran):
        if train_predictions[i] == y_train[i]:
            correct+=1
    print 'train accuracy: ', float(correct)/len(y_train)
    test_predictions = clf.predict(X_test)
    correct = 0
    for i in range(0,ran):
        if test_predictions[i] == y_test[i]:
            correct+=1
    print 'test accuracy: ', float(correct)/len(y_test)

Question 6:
XY ready
c =  1000
train accuracy:  0.9226
test accuracy:  0.85632


In [ ]:
    print 'Question 7:'
    ran = 25000
    c = 10000000
    X = list()
    Y = list()
    for d in data:
        X.append([d['review/taste'],len(d['beer/name'])])
        if d['beer/style'] == 'American IPA':
            Y.append(1)
        else:
            Y.append(0)
    print 'XY ready'
    X_train = X[:ran]
    y_train = Y[:ran]
    X_test = X[ran:2*ran]
    y_test = Y[ran:2*ran]
    # Create a support vector classifier object, with regularization parameter C = 1000
    clf = svm.SVC(C=c)
    print 'c = ',c
    clf.fit(X_train, y_train) 
    train_predictions = clf.predict(X_train)
    correct = 0 
    for i in range(0,ran):
        if train_predictions[i] == y_train[i]:
            correct+=1
    print 'train accuracy: ', float(correct)/len(y_train)
    test_predictions = clf.predict(X_test)
    correct = 0
    for i in range(0,ran):
        if test_predictions[i] == y_test[i]:
            correct+=1
    print 'test accuracy: ', float(correct)/len(y_test)

Question 7:
XY ready
c =  10000000


In [4]:
(0 and 0)

0